# Baseline

В конкурсе представлены реальные данные объявлений о продаже товаров с сайта Avito.
Сылка на датасет: 
https://www.kaggle.com/datasets/vitaliy3000/avito-dataset

Для удобства пользователей каждое объявление размещается в одной из категорий в зависимости от типа продаваемого товара. 

Всего в выборке 50 различных категорий.

Вам предлагается построить алгоритм, который на основе текстов объявлений будет предсказывать категорию товара.





Загрузим данные и посмотрим на них

In [ ]:
import pandas as pd

In [ ]:
Train = pd.read_csv('train.csv')

In [ ]:
Test = pd.read_csv('test.csv')

In [ ]:
Train.head()

,title,description,Category_name,Category
0,"Эбу Renault / Nissan 1,5 DCI Delphi 2001-2008 год","Комплект ЭБУ (мозги, компьютер мотора, двигате...",Запчасти и аксессуары,10
1,Утюг утп 1000 ватт СССР 1987 год,Продам/\n Фото № 1-2 /\n /\nУтюг УТП 1000 ватт...,Бытовая техника,21
2,Возвму машину с выкупом,Возьму машину в аренду с последующим выкупом н...,Предложение услуг,114
3,Полусапожки,"полусапожки в отличном состоянии, один раз оде...","Одежда, обувь, аксессуары",27
4,Босоножки кожаные,"Кожаные(натур) босоножки Karlo Pasolini, 40 рр...","Одежда, обувь, аксессуары",27


In [ ]:
Test.head()

,title,description,itemid,title&description
0,Мастерка,Мастерка фирмы форвард. Белого цвета. В идеаль...,1778449823,Мастерка Мастерка фирмы форвард. Белого цвета....
1,Зимние сапоги,"Продаю зимние сапоги, в хорошем состоянии, все...",1677656962,"Зимние сапоги Продаю зимние сапоги, в хорошем ..."
2,Видеонаблюдение 8 камер,В комплект Atis AMD-2MIR-8kit входит: /\n1. Ку...,1758182804,Видеонаблюдение 8 камер В комплект Atis AMD-2M...
3,Запчасти для GLE,Запчасти GLE,1689811299,Запчасти для GLE Запчасти GLE
4,Бластер nerf,Состояние 5+/\nПродаю потому что не нужен/\n18...,1804706240,Бластер nerf Состояние 5+/\nПродаю потому что ...


In [ ]:
Train.isnull().sum()

title            0
description      2
Category_name    0
Category         0
dtype: int64

In [ ]:
Test.isnull().sum()

title          0
description    1
itemid         0
dtype: int64

Видно, что есть объекты с пропуском в текстовом поле `description`. Заменим пропуски на пустую строку

In [ ]:
Train.fillna('', inplace=True)

In [ ]:
Test.fillna('', inplace=True)

Для простоты конкатенируем строки из полей `title` и `description`

In [ ]:
Train['title&description'] = Train['title'].str[:] + ' ' + Train['description'].str[:]

In [ ]:
Test['title&description'] = Test['title'].str[:] + ' ' + Test['description'].str[:]

Разделим выборку ```Train``` на обучающую и тестовую

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Train[['title&description']], Train['Category'])


Применим ```Tfidf``` преобразование к текстам, чтобы получить признаковое описание каждого объявления

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tf_idf = TfidfVectorizer()
tf_idf.fit(X_train['title&description'])

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.float64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [ ]:
X_train_tf_idf = tf_idf.transform(X_train['title&description'])
X_test_tf_idf = tf_idf.transform(X_test['title&description'])

Посмотрим на полученные размеры матриц объекты-признаки

In [ ]:
X_train_tf_idf.shape

(3175531, 1785706)

In [ ]:
X_test_tf_idf.shape

(1058511, 1785706)

Обучим линейный классификатор на полученных данных

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

In [ ]:
clf = SGDClassifier()
clf.fit(X_train_tf_idf, y_train)


/Users/aarysmyatova/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

Посчитаем качество предсказания по метрике ``Accuracy``

In [ ]:
accuracy_score(y_test, clf.predict(X_test_tf_idf))

0.8473346049308887

Подготовим файл с предсказаниями категорий для `Test.csv` 

In [ ]:
tf_idf = TfidfVectorizer()
tf_idf.fit(Train['title&description'])

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.float64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [ ]:
Train_tf_idf = tf_idf.transform(Train['title&description'])
Test_tf_idf = tf_idf.transform(Test['title&description'])

In [ ]:
clf = SGDClassifier()
clf.fit(Train_tf_idf, Train['Category'])

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
Answer = pd.DataFrame(columns=['Id', 'Category'])
Answer['Id'] = Test['itemid']

In [ ]:
Answer['Category'] = clf.predict(Test_tf_idf)

In [ ]:
Answer.to_csv('my_submission.csv', index=None)

Что можно сделать дальше?
 - попробуйте обработать текстовые данные, чтобы снизить размерность пространства признаков (стэмминг, лемматизация, удаление стоп-слов)
 - экспериментируйте с использованием других классификаторов
 - настройте гиперпараметры классификатора
 - примените стэкинг классификаторов
 - исследуйте данный и найдите интересные закономерности